In [1]:
from __future__ import division
%pylab inline
import seaborn as sns;sns.set_style('darkgrid')
%config InlineBackend.figure_format = 'retina'
matplotlib.rcParams['figure.figsize'] = 19,14
sns.set_context('poster',font_scale=1.4)
import galsim as gs

Populating the interactive namespace from numpy and matplotlib


In [6]:
cat = gs.COSMOSCatalog(use_real=False)

In [7]:
cat.getNObjects()

81520

In [45]:
sigmas = []
for idx in range(cat.getNObjects()):
    galaxy = cat.makeGalaxy(idx)
    try:
        img = galaxy.drawImage()
        moments = gs.hsm.FindAdaptiveMom(img)
        sigmas.append(moments.moments_sigma)
    except RuntimeError:
        print('skipping this one')


skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping t

skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping t

skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping t

skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping t

skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping t

...

skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping t

skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping t

skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping t

skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping t

skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping this one
skipping t

In [47]:
len(sigmas)

26754

In [58]:
print('fraction of useable galaxies: {:.3f}'.format(len(sigmas)/cat.getNObjects()))

fraction of useable galaxies: 0.328


In [48]:
sigmas = np.array(sigmas)

In [59]:
np.mean(sigmas)

7.2891019744603742

In [49]:
traces = 2*sigmas**2

In [50]:
np.mean(traces)

145.55015019085121

## Second attempt:
- Use Kolmogorov profile for several realizations of the psf simulating a distribution similar to the one in page 48 of https://docushare.lsstcorp.org/docushare/dsweb/Get/LPM-17, add 10% to the size, draw them with some small pizel scale (0.02) and get moments and invert pixel scale.
- Use number from documentation. 

In [ ]:
psf = gs.Kolmogorov(fwhm=)
